# Background

It is recommended by the Center for Disease Control and other health organizations that individuals do at least 150 minutes of moderate intensity activity a week, or 75 minutes of vigorous intensity activity a week (CDC). This activity can be spread evenly over 4-5 days a week. However, data collected from the CDC shows that only about 23% of all U.S. adults get the recommended amount of exercise per week. Why is this the case? Is it due to an aversion to exercise, lack of time to exercise, or a lack of opportunity? Surveys conducted by various fitness apps such as Freeletics found that “not having time” is the number one barrier to exercise that Americans face, with 42% of participants saying that time is the biggest reason they fail to work out. Thus, it would be beneficial to find an exercise routine that is time-effective and efficient in elevating heartrate, burning calories and hitting as many muscle groups in the body as possible. 

One largely favored solution amount physical trainers and fitness gurus alike is high-intensity interval training (HIIT). HIIT workouts have grown in popularity over the past few years because of how accessible they are – HIIT exercises are incredibly effective using just bodyweight, and can be done almost anywhere. There’s no requirement for a gym, weights, or specific equipment in order to do the workout. HIIT exercises are also efficient; they are successful in elevating the heartrate and burning calories over a short period of time (Gibala 2007). However, HIIT is quite an intense workout and can be a daunting challenge for those who don’t regularly exercise. Our goal is to investigate if HIIT is truly more effective than moderate intensity training such as jogging. A comparison between HIIT and moderate intensity continuous training will be done as a reality check to see what the difference between the two training forms is. 

# Hypothesis

High-intensity interval training will be more effective in increasing heart rate and burning calories than moderate-intensity continuous training. 

# Protocol

### Part I: Gathering baseline measurements 

Before the participant begins the exercises, their baseline attributes must be measured and recorded in the “resting” row of the spreadsheet. These measurements should be recorded in a restful state, which involves the participant to rest for 60 seconds before measurements are taken. The required measurements are listed below: 

    -Heart Rate: use fitbit 
    -Steps taken: use fitbit 
    -Calories burned: use fitbit 
    -Respiratory Rate: count number of breaths for 15 seconds and multiply by 4 
    -Likert Scale of Exhaustion 
        -0 = less than usual 
        -1 = no more than usual 
        -2 = worse than usual 
        -3 = much worse than usual 
    -Time start: use 2400 hours scale, start right at whole minute 
    -Time end: use 2400 hours scale, start right at whole minute 
    -Height: in inches 
    -Weight: in lbs 


### Part II: HIIT vs moderate intensity continuous training 

**Sprint (high-intensity interval training)**

Each participant will do 5 sets of minute-long sprints (20 secs on, 10 secs off) on a treadmill, which will be representative of high-intensity interval training. They will begin their sprints on the minute, recording what time they’re starting at. They will then do a minute of exercise (so two rounds of 20 secs on, 10 secs off). Once the minute is up, they will record their measurements (see above) immediately, and rest until they feel they have recovered. Sprints during the on-period should be at an effort level of 10/10, with speeds dependent on participants. The off-period should consist of walking at an active pace.  

    -Head to treadmills in gym.
    -Run on treadmill for 1 minute on an interval of 20 seconds of sprinting and 10 seconds of walking
    -Make sure to start sprints on the minute and to record that time
    -After minute is us, record measurements above immediately
    -Rest until participant feels recovered (suggestion: 2-3 minutes)
    -Repeat process 4 more times

**Jogging (moderate intensity continuous training)**

The participant will do 5 sets of minute-long jogging at a comfortable pace (between 7.0 and 8.0 mph) on a treadmill. They will be their jogging set on the minute, recording what time they’re starting at. Once the minute of exercise has been completed, they will record the time at which the exercise was completed as well as all measurements listed above. 

    -Head to treadmills in gym.
    -Run on treadmill for 1 minute, jogging at a comfortable pace (suggestion: 7.0-8.0 mph)
    -Make sure to start sprints on the minute and to record that time
    -After minute is us, record measurements above immediately
    -Rest until participant feels recovered (suggestion: 2-3 minutes)
    -Repeat process 4 more times

### Data Collection

The participant will fill in a provided spreadsheet with all the necessary information. After all exercises have been completed and data has been conducted, save the sample spreadhseet (with subsequent data filled in) as "hiit_data_*lastname*" and as a csv file. Then upload this file into the provided Box folder. This file will be used further in this lab. Below is a link to the spreadsheet.

https://wlu.box.com/s/h3orvz5sfbj9984jgk06xa66i4jvvq9p

# Data Extraction

In this section, we will be replicated what we did in Lab 6, where we extract the data from the Fitbit and combine it with the data that was manually collected.

Below are some packages and import statements necessary to complete this process.

In [ ]:
import sys
!{sys.executable} -m pip install cherrypy
!{sys.executable} -m pip install fitbit

In [ ]:
import fitbit
import gather_keys_oauth2 as Oauth2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import pandas as pd
import datetime
import os
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot as plt
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Now, create variables for your client id and client secret taken from the final screen in creating your fitbit application.

In [ ]:
CLIENT_ID = 
CLIENT_SECRET = 

This code will give you access and be authenticated by the server.

In [ ]:
server = Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN = str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN = str(server.fitbit.client.session.token['refresh_token'])
auth2_client = fitbit.Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN)

The following function works to extract all the necessry data from your fitbit. Please fill in the values 'daysAgo', 'start_time', and 'end time' with the appropriate information.

In [ ]:
def extractData(auth2_client, daysAgo):
    """
    Extracting data through the authorized client. Looking for access to different
    activities at different granularities of time.
    Input: Authorized client and file name
    Output: Output csv file with necessary information
    """

    features = ["activities/heart", "activities/steps", "activities/calories"]
    daysAgo = 0
    features_intraday = ["activities-heart-intraday", "activities-steps-intraday", "activities-calories-intraday"]
    dateOfInterest = str((datetime.datetime.now() - datetime.timedelta(days=daysAgo)).strftime("%Y-%m-%d"))  
    count = 0

    column_names = ["Time", "Heart Rate", "Steps", "Calories"]
    df = pd.DataFrame(columns = column_names)

    for feature in features:
        fit_stats = auth2_client.intraday_time_series(
                              feature,
                              base_date = dateOfInterest,
                              detail_level = '1min',
                              start_time   = '00:00',
                              end_time     = '00:00')
        temp_value = []
        temp_time = []

        for i in fit_stats[features_intraday[count]]['dataset']:
            temp_value.append(i['value'])
            temp_time.append(i['time'])
        df["Time"] = temp_time
        df.iloc[:, count+1] = temp_value
        count += 1

  
    display(df)
    df.to_csv(os.getcwd() + '\\' + "allfeatures" + dateOfInterest + '.csv', columns=['Time', 'Heart Rate', 'Steps', 'Calories'], header=True, index = False)

In [ ]:
extractData(auth2_client, 1)

Now we need to work on combining the data that we manually collected with out fitbit data. Read in your file that you saved before as 'hiit_data'.

In [ ]:
hiit_data = pd.read_csv()
hiit_data

Next, read in our fitbit data as df with the appropriate file name.

In [ ]:
df = pd.read_csv()
df

You will need to manipulate the Time column of the fitbit data such that it has the same format as what you see in the manually collected data Time Start column.

In [ ]:
def change_to_number(val):
    hour = val[0:2]
    minute = val[3:5]
    time = hour + minute
    return time

In [ ]:
military = []
for i in df.Time:
    hour = i[0:2]
    minute = i[3:5]
    time = hour+minute
    military.append(time)
    
df["Time"] = military
df

After this manipulation of the fitbit dataframe. You need add the features from both dataframes together into one, while making sure that the Time Start column from your data and the Time value from the fitbit dataframe match. This way, you are getting the fibit features from the exact time you were doing your exercise.

In [ ]:
partial_df = pd.DataFrame()

times = []

df['Time'] = df['Time'].astype(str)

lab5Data['Time Start'] = lab5Data['Time Start'].astype(str)

for i in range(len(lab5Data['Time Start'])):
    times.append(lab5Data['Time Start'][i])
    
partial_df = df.loc[df['Time'].isin(times)]

final_df = pd.merge(left=lab5Data, right=partial_df, how='left', left_on='Time Start', right_on='Time')

final_df

Next, remove the Time Start column from your dataframe. Lastly, write this whole dataframe to a csv file named fitbit_data_yourlastname. Then please email this file to either ardoinm24@mail.wlu.edu or kolevai22@mail.wlu.edu.

In [ ]:
final_df.drop(["Time Start"], axis = 1, inplace = True)
final_df.to_csv(, header=True, index = False)